# Padding

- 자연어 처리에서 각 문장(문서)의 길이는 서로 다를 수 있음
- 하지만 대부분의 모델은 고정 길이 입력을 기준으로 배치(batch) 단위 학습을 효율적으로 수행함
- 따라서 모든 문장의 길이를 동일한 길이(maxlen) 로 맞춰주는 작업이 필요함 → Padding

**Padding 개념**
- Padding: 짧은 문장에 PAD 같은 특수 토큰(보통 0) 을 채워 길이를 맞춤
- Truncation(잘라내기): 너무 긴 문장은 maxlen 기준으로 일정 길이까지만 남기고 자름
- Padding 방향
    - post padding: 뒤에 채움(일반적으로 많이 사용)
    - pre padding: 앞에 채움(모델/설정에 따라 사용)

**사용시기**
- 문장/문서를 시퀀스(정수 인덱스)로 바꾼 뒤 모델 입력으로 넣을 때
    - 예: Embedding + RNN/LSTM/GRU, 1D CNN, Transformer 계열 등
- 미니배치 학습(DataLoader/fit)에서 텐서 크기를 맞춰야 할 때
    - 배치로 묶으려면 (batch, seq_len) 형태로 길이가 동일해야 함
- 평가/추론에서도 동일하게 적용
    - 학습 때 사용한 maxlen 기준으로 테스트/서비스 입력도 동일 처리 필요

**코드 내 사용 위치**
- 텍스트 정제(소문자화/특수문자 처리 등)
- 토큰화(단어/서브워드)
- 정수 인코딩(Tokenizer, vocab 매핑)
- Padding/Truncation 적용
- 모델 입력(Embedding/Encoder) → 학습/평가

**Padding 이점**
- 일관된 입력 형식: 모든 문장이 동일한 길이의 시퀀스로 변환되어 모델 입력이 단순해짐
- 병렬 연산 최적화: 배치 단위 텐서 연산이 가능해져 GPU/행렬 연산 효율이 좋아짐
- 유연한 데이터 처리: 다양한 길이의 문서를 동일한 파이프라인으로 처리 가능

**주의사항(중요)**
- PAD는 의미 없는 값이므로 학습에 영향을 주면 안 됨
    - Masking(마스킹) 으로 PAD 위치를 손실 계산/어텐션 계산에서 제외하는 경우가 많음
- maxlen을 너무 크게 잡으면 PAD가 과도하게 늘어 연산 낭비 + 성능 저하가 생길 수 있음
    - 문장 길이 분포를 보고 적절한 maxlen을 선택하는 것이 좋음

In [29]:
!pip3 install torch torchvision

In [ ]:
preprocessed_sentences = [
    ['barber', 'person'],
    ['barber', 'good', 'person'],
    ['barber', 'huge', 'person'],
    ['knew', 'secret'],
    ['secret', 'kept', 'huge', 'secret'],
    ['huge', 'secret'],
    ['barber', 'kept', 'word'],
    ['barber', 'kept', 'word'],
    ['barber', 'kept', 'secret'],
    ['keeping', 'keeping', 'huge', 'secret', 'driving', 'barber', 'crazy'],
    ['barber', 'went', 'huge', 'mountain']
]   # 문장별로 토큰화/정제된 결과를 리스트로 저장

In [ ]:
import torch
from collections import Counter # 단어 수 세기

class TokenizerForPadding:
    def __init__(self, num_words=None, oov_token='<OOV>'):
        self.num_words = num_words          # 사용할 최대 단어 수 (어휘 제한), None 이면 제한 없음
        self.oov_token = oov_token          # 사전에 없는 단어를 치환할 OOV 토큰 문자열
        self.word_index = {}                # 단어 -> 인덱스 매핑 딕셔너리
        self.index_word = {}                # 인덱스 -> 단어 매핑 딕셔너리
        self.word_counts = Counter()        # 단어 빈도수
        
    # 단어사전 생성 및 단어 -> 인덱스, 인덱스 -> 단어 딕셔너리 생성하는 함수
    def fit_on_texts(self, texts):
        # 빈도수 세기
        for sentence in texts:      # 문장(토큰 리스트) 단위로 순회 
            self.word_counts.update(word for word in sentence if word)           # 빈 문자열 제외하고 단어 빈도 누적
            
        # 빈도수 기반으로 vocabulary 생성
        vocab = [self.oov_token] + [word for word, _ in self.word_counts.most_common(self.num_words - 2 if self.num_words else None)]   # 빈도 상위 단어
        
        self.word_index = {word: i+1 for i, word in enumerate(vocab)}           # 1번(<OOV>)부터 단어 -> 인덱스
        self.index_word = {i: word for word, i in self.word_index.items()}      # 인덱스 -> 단어 역매핑
        
    def texts_to_sequences(self, texts):
        return [[self.word_index.get(word, self.word_index[self.oov_token]) for word in sentence ] for sentence in texts]   

vocab = [self.oov_token] + [word for word, _ in self.word_counts.most_common(self.num_words - 2 if self.num_words else None)]  
[slef.oov_token] : OOV 토큰을 voca 의 첫 원소로 추가  
self.word_counts.most_common(k) : 등장 횟수가 많은 순서대로 (단어, 빈도) 튜플을 k개 반환  
self.num_words - 2 if self.num_words else None : num_words를 지정했으면 상위단어로 num_words -2개 반환  
-2의 이유는 하나(0)는 padding 용도, 다른 하나는 <OOV> 용도로 자리 2개를 비워놓는다.

In [32]:
# 시퀀스 Padding/truncation 함수
def pad_sequences(sequences, maxlen=None, padding='pre', truncating='pre', value=0):
    if maxlen is None:                                  # 최대 길이를 따로 설정안하면
        maxlen = max(len(seq) for seq in sequences)     # 입력 시퀀스들 중 가장 긴 길이를 maxlen으로 사용
        
    padded_sequences = []                               # 패딩/트렁케이션 적용 결과 리스트
    for seq in sequences:
        if len(seq) > maxlen:                           # 시퀀스가 maxlen보다 길면 (Truncation)
            if truncating == 'pre':                     # pre 인 경우
                seq = seq[-maxlen:]                     # seq는 뒤에서 maxlen개만 남김
            else :                                      # post인 경우
                seq = seq[:maxlen]                      # 앞에서 maxlen개를 남김
        else:                                           # 시퀀스가 maxlen보다 작으면 (padding)
            pad_length = maxlen - len(seq)              # 패딩 길이 계산
            
            if padding =='pre':                         # pre인 경우 
                seq = [value] * pad_length + seq        # 앞쪽에 0 패딩을 추가
            else:
                seq = seq + [value] * pad_length        # 뒤쪽에 0 패딩을 추가 (원본 + PAD ....)
        padded_sequences.append(seq)                    # 처리된 시퀀스를 결과 리스트에 추가
    return torch.tensor(padded_sequences)               # 최종 결과를 Pythorch 텐서로 변환해 반환
        

반환값은 (문장개수, maxlen) 형태의 2차원 텐서.  
짧은 시퀀스는 value로 채워지고 (padding), 긴 시퀀스는 truncating 기준으로 잘린다.

In [33]:
tokenizer = TokenizerForPadding(num_words=15)
tokenizer.fit_on_texts(preprocessed_sentences)
sequences = tokenizer.texts_to_sequences(preprocessed_sentences)
sequences 

[[2, 6],
 [2, 9, 6],
 [2, 4, 6],
 [10, 3],
 [3, 5, 4, 3],
 [4, 3],
 [2, 5, 7],
 [2, 5, 7],
 [2, 5, 3],
 [8, 8, 4, 3, 11, 2, 12],
 [2, 13, 4, 14]]

In [34]:
# 시퀀스 길이를 3으로 맞추는 패딩/트렁케이션 적용
padded = pad_sequences(sequences, padding="post", truncating="pre", maxlen=3)   # 길이가 3보다 뒤(post)에 0 패딩, 길면 앞(pre)을 버리고 1

- 예제처럼 긴문장의 뒤를 남길 
    - 핵심 정보가 문장 끝에 몰리는 데이터가 몰리는 경우
        - 리뷰/감정: “... but I hated it”, “... not good” 처럼 결론이 뒤에 오는 경우
        - 질의/명령: “I want to book a ticket tomorrow” 같은 핵심 슬롯이 뒤에 붙는 경우
    - RNN/LSTM 처럼 순차적으로 보는 모델은 마지막 쪽 토큰 영향이 더 크게 남는 경우가 많음
- 앞을 남길 때 (post truncating)
    - 뉴스/보고서처럼 서론(앞부분)에 핵심이 많은 데이터
    - 제목/헤드라인 기반 과제
    - 길이가 길어도 초반 컨텍스트가 중요한 태스크

- 정답은 없음 → 데이터 특성에 맞춰 선택. 보통은 둘 다 돌려보고 성능 좋은 쪽 선택
- 실제로는 maxlen을 충분히 크게 잡고(예: 50/100/200) truncation 정책을 정한다.
- Transformer(BERT류)는 보통 앞(초반) 유지를 더 많이 쓰는 편

| 구분 | 어떤 걸 남김 | 언제 유리한가(데이터/태스크) | 예시 | 모델 관점/메모 |
|---|---|---|---|---|
| **뒤를 남김** | **문장 끝(뒤쪽) 유지** *(= 앞을 잘라냄, pre-truncating)* | 핵심 정보가 **결론/마지막**에 몰리는 데이터 | 리뷰/감정: `... but I hated it`, `... not good` / 질의·명령: `... book a ticket tomorrow` | RNN/LSTM처럼 순차 모델은 **후반 토큰 영향**이 크게 남는 경우가 많음 |
| **앞을 남김** | **문장 시작(앞쪽) 유지** *(= 뒤를 잘라냄, post-truncating)* | 핵심이 **서론/초반**에 몰리는 데이터 | 뉴스/보고서 서두 / 제목·헤드라인 기반 과제 | Transformer(BERT류)는 실무에서 **앞부분 유지**를 더 자주 쓰는 편 |
| **공통 원칙** | — | 정답은 없고 **데이터 특성에 맞게 선택** | 보통 두 정책 다 돌려보고 성능 좋은 쪽 선택 | `maxlen`을 충분히 크게(예: 50/100/200) 잡고 truncation 정책을 결정하는 경우가 많음 |

### Keras Tokenizer 사용

In [35]:
preprocessed_sentences = [
    ['barber', 'person'],
    ['barber', 'good', 'person'],
    ['barber', 'huge', 'person'],
    ['knew', 'secret'],
    ['secret', 'kept', 'huge', 'secret'],
    ['huge', 'secret'],
    ['barber', 'kept', 'word'],
    ['barber', 'kept', 'word'],
    ['barber', 'kept', 'secret'],
    ['keeping', 'keeping', 'huge', 'secret', 'driving', 'barber', 'crazy'],
    ['barber', 'went', 'huge', 'mountain']
]   # 문장별로 토큰화/정제된 결과를 리스트로 저장

In [36]:
from tensorflow.keras.preprocessing.text import Tokenizer # keras 토크나이저 (단어 인덱싱/시퀀스 변환)

tokenizer = Tokenizer()                             # 기본 설정 (단어 개수 제한 없음, OOV 토큰 미지정)
tokenizer.fit_on_texts(preprocessed_sentences)      # 단어 빈도 기반 인덱스(word_index) 생성
sequences = tokenizer.texts_to_sequences(preprocessed_sentences)    # 문장(토큰 리스트)를 정수 인덱스로 변환
sequences



[[1, 5],
 [1, 8, 5],
 [1, 3, 5],
 [9, 2],
 [2, 4, 3, 2],
 [3, 2],
 [1, 4, 6],
 [1, 4, 6],
 [1, 4, 2],
 [7, 7, 3, 2, 10, 1, 11],
 [1, 12, 3, 13]]

In [37]:
tokenizer.word_index

{'barber': 1,
 'secret': 2,
 'huge': 3,
 'kept': 4,
 'person': 5,
 'word': 6,
 'keeping': 7,
 'good': 8,
 'knew': 9,
 'driving': 10,
 'crazy': 11,
 'went': 12,
 'mountain': 13}

In [38]:
tokenizer.index_word

{1: 'barber',
 2: 'secret',
 3: 'huge',
 4: 'kept',
 5: 'person',
 6: 'word',
 7: 'keeping',
 8: 'good',
 9: 'knew',
 10: 'driving',
 11: 'crazy',
 12: 'went',
 13: 'mountain'}

In [39]:
# tokenizer.index_word
tokenizer.index_word[1]

'barber'

In [40]:
# Keras pad_sequences 로 길이를 맞추기
from tensorflow.keras.preprocessing.sequence import pad_sequences

padded = pad_sequences(sequences, padding='post', truncating='post', maxlen=3)

padded

array([[ 1,  5,  0],
       [ 1,  8,  5],
       [ 1,  3,  5],
       [ 9,  2,  0],
       [ 2,  4,  3],
       [ 3,  2,  0],
       [ 1,  4,  6],
       [ 1,  4,  6],
       [ 1,  4,  2],
       [ 7,  7,  3],
       [ 1, 12,  3]], dtype=int32)

### [연습] 어린왕자 데이터 샘플 패딩처리

1. 텍스트 전처리 (토큰화/불용어처리/정제/정규화)
2. 정수 인코딩 by Tokenizer (tensorflow.keras)
3. 패딩 처리 by pad_sequences (tensorflow.keras)

In [41]:
raw_text = """The Little Prince, written by Antoine de Saint-Exupéry, is a poetic tale about a young prince who travels from his home planet to Earth. The story begins with a pilot stranded in the Sahara Desert after his plane crashes. While trying to fix his plane, he meets a mysterious young boy, the Little Prince.

The Little Prince comes from a small asteroid called B-612, where he lives alone with a rose that he loves deeply. He recounts his journey to the pilot, describing his visits to several other planets. Each planet is inhabited by a different character, such as a king, a vain man, a drunkard, a businessman, a geographer, and a fox. Through these encounters, the Prince learns valuable lessons about love, responsibility, and the nature of adult behavior.

On Earth, the Little Prince meets various creatures, including a fox, who teaches him about relationships and the importance of taming, which means building ties with others. The fox's famous line, "You become responsible, forever, for what you have tamed," resonates with the Prince's feelings for his rose.

Ultimately, the Little Prince realizes that the essence of life is often invisible and can only be seen with the heart. After sharing his wisdom with the pilot, he prepares to return to his asteroid and his beloved rose. The story concludes with the pilot reflecting on the lessons learned from the Little Prince and the enduring impact of their friendship.

The narrative is a beautifully simple yet profound exploration of love, loss, and the importance of seeing beyond the surface of things."""

In [42]:
# NLTK 전처리 : 문장/단어 토큰화 + 불용어/짧은 토큰 제거 + 단어 빈도 사전
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords

# 문장 토큰화
sentences = sent_tokenize(raw_text)

# 영어 불용어 리스트
en_stopwords = stopwords.words('english')

# 단어사전 (key=단어, value=빈도)
vocab={}

# 토큰화/정제/정규화 처리 결과
preprocessed_sentences = []

for sentence in sentences:
    sentence = sentence.lower()                                             # 소문자 변환
    tokens = word_tokenize(sentence)                                        # 문장을 단어(토큰) 리스트로 변환
    tokens = [token for token in tokens if token not in en_stopwords]       # 불용어 토큰 제거
    tokens = [token for token in tokens if len(token)>2]                    # 길이 2 이하 토큰 제거 (노이즈 감소)
    
    # 전처리된 토큰들로 빈도 집계
    for token in tokens:
        # vocab에 없으면 빈도 1로 초기화, vocab에 이미 존재하면 +1
        if token in vocab:
            vocab[token] += 1 
        else :
            vocab[token] = 1

    preprocessed_sentences.append(tokens)

preprocessed_sentences : 문장별 토큰 리스트  
vocab : 전체 단어의 빈도 딕셔너리

In [44]:
# Keras Tokenizer : 상위 단어 제한 + OOV 처리 + 정수 시퀀스 변환
from tensorflow.keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(num_words=15, oov_token='<OOV>')  # 상위 15개 단어, oov 토큰은 '<OOV>'로 사용
            
tokenizer.fit_on_texts(preprocessed_sentences)                       # 코퍼스에서 단어 빈도 기반 인덱스 사전 생성

sequences = tokenizer.texts_to_sequences(preprocessed_sentences)     # 문장(토큰 리스트)을 정수 인덱스 시퀀스로 변환

sequences

[[3, 2, 1, 1, 1, 1, 1, 7, 2, 1, 1, 8, 9],
 [10, 1, 4, 1, 1, 1, 11, 1],
 [1, 1, 11, 12, 1, 7, 1, 3, 2],
 [3, 2, 1, 1, 13, 1, 1, 1, 1, 5, 1, 1],
 [1, 1, 4, 1, 1, 1, 1],
 [8, 1, 1, 1, 1, 1, 1, 1, 1, 1, 6],
 [1, 2, 1, 1, 14, 1, 1, 1, 1, 1],
 [9, 3, 2, 12, 1, 1, 1, 6, 1, 1, 1, 1, 1, 1, 1, 1],
 [6, 1, 1, 1, 1, 1, 1, 1, 2, 1, 5],
 [1, 3, 2, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 4, 1, 1, 13, 1, 5],
 [10, 1, 4, 1, 14, 1, 3, 2, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]

In [48]:
# 시퀀스 패딩/길이 맞춤 유틸 함수
from tensorflow.keras.preprocessing.sequence import pad_sequences

# 기본옵션 : pre padding, maxlen=최장길이, value = 0
padded = pad_sequences(sequences)       # 기본옵션 : pre padding, maxlen=최장길이, value = 0
padded  # (문장 개수, 최장길이) 형태의 2D 배열

array([[ 0,  0,  0,  3,  2,  1,  1,  1,  1,  1,  7,  2,  1,  1,  8,  9],
       [ 0,  0,  0,  0,  0,  0,  0,  0, 10,  1,  4,  1,  1,  1, 11,  1],
       [ 0,  0,  0,  0,  0,  0,  0,  1,  1, 11, 12,  1,  7,  1,  3,  2],
       [ 0,  0,  0,  0,  3,  2,  1,  1, 13,  1,  1,  1,  1,  5,  1,  1],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  1,  4,  1,  1,  1,  1],
       [ 0,  0,  0,  0,  0,  8,  1,  1,  1,  1,  1,  1,  1,  1,  1,  6],
       [ 0,  0,  0,  0,  0,  0,  1,  2,  1,  1, 14,  1,  1,  1,  1,  1],
       [ 9,  3,  2, 12,  1,  1,  1,  6,  1,  1,  1,  1,  1,  1,  1,  1],
       [ 0,  0,  0,  0,  0,  6,  1,  1,  1,  1,  1,  1,  1,  2,  1,  5],
       [ 0,  0,  0,  0,  0,  0,  1,  3,  2,  1,  1,  1,  1,  1,  1,  1],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  1,  1,  4,  1,  1, 13,  1,  5],
       [ 0,  0,  0,  0,  0, 10,  1,  4,  1, 14,  1,  3,  2,  1,  1,  1],
       [ 0,  0,  0,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1]],
      dtype=int32)